In [1]:
import sys
import importlib
import logging
import numpy as np

sys.path.append("./src/")
import pdb_numpy
import pdb_numpy.alignement as alignement
import pdb_numpy.DSSP as DSSP


In [2]:
test = pdb_numpy.Coor(pdb_id="3eam")
test.write_pdb("./tmp.pdb", check_file_out=False)


In [3]:
print('Adding NH')
DSSP.add_NH(test)
print('NH added')

Adding NH
NH added


In [3]:
# run dssp:
# sudo apt install dssp
# time dssp -i tmp.pdb -o tmp_dssp.txt

with open("./tmp_dssp.txt") as f:
    lines = f.readlines()

start_read = False
dssp_seq = ""
for line in lines:
    if line.startswith("  #  RESIDUE AA STRUCTURE BP1 BP2  ACC"):
        start_read = True
        continue
    if start_read and line[13] != "!":
        dssp_seq += line[16]


In [5]:
%timeit DSSP.add_NH(test)

989 ms ± 109 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit DSSP.get_NH_xyz(test)

36.4 ms ± 1.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%timeit SS_list = DSSP.compute_DSSP(test)

867 ms ± 40.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%load_ext line_profiler

In [9]:
%lprun -f DSSP.compute_Hbond_matrix DSSP.compute_DSSP(test)

Timer unit: 1e-09 s

Total time: 0.444893 s
File: ./src/pdb_numpy/DSSP.py
Function: compute_Hbond_matrix at line 203

Line #      Hits         Time  Per Hit   % Time  Line Contents
   203                                           def compute_Hbond_matrix(model):
   204                                               """Compute Hbond matrix for a protein.
   205                                               """
   206         1       1518.0   1518.0      0.0      cutoff = 8.0
   207                                           
   208         1    5134739.0 5134739.0      1.2      CA_array = model.select_atoms("protein and name CA and not altloc B C D E").xyz
   209         1       2211.0   2211.0      0.0      n_res = len(CA_array)
   210                                           
   211         1   87669937.0 87669937.0     19.7      dist_mat = distance_matrix(CA_array, CA_array)
   212         1    1114830.0 1114830.0      0.3      Hbond_mat = np.zeros_like(dist_mat, dtype=bool)
   213   

In [14]:
test = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
np.where(test == 4)[0][0]

3

In [2]:
pdb_numpy.show_log()

In [3]:
importlib.reload(pdb_numpy)

<module 'pdb_numpy' from './src/pdb_numpy/__init__.py'>

In [4]:
test = pdb_numpy.Coor('./src/pdb_numpy/tests/input/2rri.pdb')

Succeed to read file src/pdb_numpy/tests/input/2rri.pdb 
479 atoms found


In [5]:
test.crystal_pack

'CRYST1    1.000    1.000    1.000  90.00  90.00  90.00 P 1           1          \n'

In [6]:
test.model[0].resname

array([b'HIS', b'HIS', b'HIS', b'HIS', b'HIS', b'HIS', b'HIS', b'HIS',
       b'HIS', b'HIS', b'HIS', b'HIS', b'HIS', b'HIS', b'HIS', b'HIS',
       b'HIS', b'HIS', b'SER', b'SER', b'SER', b'SER', b'SER', b'SER',
       b'SER', b'SER', b'SER', b'SER', b'SER', b'ASP', b'ASP', b'ASP',
       b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP',
       b'ASP', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA',
       b'ALA', b'ALA', b'ALA', b'VAL', b'VAL', b'VAL', b'VAL', b'VAL',
       b'VAL', b'VAL', b'VAL', b'VAL', b'VAL', b'VAL', b'VAL', b'VAL',
       b'VAL', b'VAL', b'VAL', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE',
       b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE',
       b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'THR',
       b'THR', b'THR', b'THR', b'THR', b'THR', b'THR', b'THR', b'THR',
       b'THR', b'THR', b'THR', b'THR', b'THR', b'ASP', b'ASP', b'ASP',
       b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP',
      

In [7]:
test = pdb_numpy.Coor('./src/pdb_numpy/tests/input/2rri.pdb')
print(test.len)
assert test.len == 479

test_CA = test.select_atoms("name CA")
print(test_CA.len)
assert test_CA.len == 29

Succeed to read file src/pdb_numpy/tests/input/2rri.pdb 
479 atoms found
479
29


In [8]:
test_CA.model[0].res_num

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [9]:
test_CA.model[0].resname = 'ALA'
test_CA.model[0].resname

array([b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA',
       b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA',
       b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA',
       b'ALA', b'ALA', b'ALA', b'ALA', b'ALA'], dtype='|S4')

In [10]:
import pdb_numpy._alignement as alignement


In [7]:

seq_1 = 'AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV\
RSGVRVKTYEPEAIWIPEIRFVNVENARDADVVDISVSPDGTVQYLERFSARVLSPLDFRRYPFDSQTLHIYLIVRSV\
DTRNIVLAVDLEKVGKNDDVFLTGWDIESFTAVVKPANFALEDRLESKLDYQLRISRQYFSYIPNIILPMLFILFISW\
TAFWSTSYEANVTLVVSTLIAHIAFNILVETNLPKTPYMTYTGAIIFMIYLFYFVAVIEVTVQHYLKVESQPARAASI\
TRASRIAFPVVFLLANIILAFLFFGF'
seq_2 = 'APSEFLDKLMGKVSGYDARIRPNFKGPPVNVTCNIFINSFGSIAETTMDYRVNIFLR\
QQWNDPRLAYSEYPDDSLDLDPSMLDSIWKPDLFFANEKGANFHEVTTDNKLLRISKNGNVLYSIRITLVLACPMDLK\
NFPMDVQTCIMQLESFGYTMNDLIFEWDEKGAVQVADGLTLPQFILKEEKDLRYCTKHYNTGKFTCIEARFHLERQMG\
YYLIQMYIPSLLIVILSWVSFWINMDAAPARVGLGITTVLTMTTQSSGSRASLPKVSYVKAIDIWMAVCLLFVFSALL\
EYAAVNFIARAGTKLFISRAKRIDTVSRVAFPLVFLIFNIFYWITY\
KLVPR'


In [8]:
importlib.reload(alignement)
%timeit align_seq_1, align_seq_2 = alignement.align_seq(seq_1, seq_2)

246 ms ± 2.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [132]:
%lprun -f alignement.align_seq alignement.align_seq(seq_1, seq_2)

Timer unit: 1e-09 s

Total time: 0.0159654 s
File: ./src/pdb_numpy/alignement.py
Function: align_seq at line 16

Line #      Hits         Time  Per Hit   % Time  Line Contents
    16                                           def align_seq_DTW(seq_1, seq_2):
    17                                               """ Align two amino acid sequences using the Dynamic Time Warping Algorithm.
    18                                               """
    19                                           
    20                                               seq_1 = seq_1.replace("-", "")
    21                                               seq_2 = seq_2.replace("-", "")
    22                                           
    23                                               len_1 = len(seq_1)
    24                                               len_2 = len(seq_2)
    25                                           
    26                                               # Initialize the matrix
    27          

In [30]:
max_index = np.argmax([3, 2, 1])
max_index

0

In [13]:
alignement.print_align_seq(align_seq_1, align_seq_2, line_len=80)

--------------AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV-RSGVRVKT
              | * |   * |   *||*| |*|| |  *| | |  ||** **  |*|* ***||    ||| |  
APSEFLDKLMGKVSGYD-ARIRPNFKGPPVNVTCNIFINSFGSIAETTMDYRVNIFLRQQWNDPRLAYSEYPDDSLDLDP


YEPEAIWIPEIRFVNVENARDADV-VD---ISVSPDGTVQYLERFSARVLSPLDFRRYPFDSQTLHIYLIVRSVDTRNIV
   ||** *|| *|* ||*|  |* |*   | |* |*|* *  *||  |  *|*||||*|* **  | *   |   ||||
SMLDSIWKPDLFFANEKGANFHEVTTDNKLLRISKNGNVLYSIRITLVLACPMDLKNFPMDVQTCIMQLESFGYTMNDLI


LAVDLE-KVGKNDDVFLTGWDIESFTAVVKPANFALEDRLESKLDYQLRISRQYFSYIPNIILPMLFILFISWTAFW-S-
|  * |  *   * | *  | |||    || ||   |    | || |||||**   *| || |* *|*|||**||** | 
FEWDEKGAVQVADGLTLPQFILKE-EKDLRYCTKHYNTGKFTCIEARFHLERQMGYYLIQMYIPSLLIVILSWVSFWINM


TSYEANVTLVVSTLIAHIAFNILVETNLPKTPYMTYTGAIIFMIYLFYFVAVIEVTVQHYL-KVESQ--PARAASITRAS
 |  *|* * ||*|||  | |   |||***| *|      | |  ** * *||* || ||| || ||   |**  *  |*
DAAPARVGLGITTVLTMTTQSSGSRASLPKVSYVKAIDIWMAVCLLFVFSALLEYAAVNFIARAGTKLFISRAKRIDTVS


RIAFPVVFLLANIILAFLFF

In [14]:
seq_1 = 'AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV\
RSGVRVKTYEPEAIWIPEIRFVNVENARDADVVDISVSPDGTVQYLERFSARVLSPLDFRRYPFDSQTLHIYLIVRSV\
DTRNIVLAVDLEKVGKNDDVFLTGWDIESFTAVVKPANFALEDRLESKLDYQLRISRQYFSYIPNIILPMLFILFISW\
TAFWSTSYEANVTLVVSTLIAHIAFNILVETNLPKTPYMTYTGAIIFMIYLFYFVAVIEVTVQHYLKVESQPARAASI\
TRASRIAFPVVFLLANIILAFLFFGF'
seq_2 = 'AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV'
align_seq_1, align_seq_2 = alignement.align_seq(seq_1, seq_2)
alignement.print_align_seq(align_seq_1, align_seq_2, line_len=80)

AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPVRSGVRVKTYEPEAIWIPEIRFVN
*********************************************************                       
AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV-----------------------


VENARDADVVDISVSPDGTVQYLERFSARVLSPLDFRRYPFDSQTLHIYLIVRSVDTRNIVLAVDLEKVGKNDDVFLTGW
                                                                                
--------------------------------------------------------------------------------


DIESFTAVVKPANFALEDRLESKLDYQLRISRQYFSYIPNIILPMLFILFISWTAFWSTSYEANVTLVVSTLIAHIAFNI
                                                                                
--------------------------------------------------------------------------------


LVETNLPKTPYMTYTGAIIFMIYLFYFVAVIEVTVQHYLKVESQPARAASITRASRIAFPVVFLLANIILAFLFFGF
                                                                             
-----------------------------------------------------------------------------


Identity seq1: 17.98%
Identit

In [15]:
seq_1 = 'AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV\
RSGVRVKTYEPEAIWIPEIRFVNVENARDADVVDISVSPDGTVQYLERFSARVLSPLDFRRYPFDSQTLHIYLIVRSV\
DTRNIVLAVDLEKVGKNDDVFLTGWDIESFTAVVKPANFALEDRLESKLDYQLRISRQYFSYIPNIILPMLFILFISW\
TAFWSTSYEANVTLVVSTLIAHIAFNILVETNLPKTPYMTYTGAIIFMIYLFYFVAVIEVTVQHYLKVESQPARAASI\
TRASRIAFPVVFLLANIILAFLFFGF'
seq_2 = 'LFYFVAVIEVTVQHYLKVESQPARAASITRASRIAFPVVFLLANIILAFLFFGF'
align_seq_1, align_seq_2 = alignement.align_seq(seq_1, seq_2)
alignement.print_align_seq(align_seq_1, align_seq_2, line_len=80)

AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPVRSGVRVKTYEPEAIWIPEIRFVN
                                                                                
--------------------------------------------------------------------------------


VENARDADVVDISVSPDGTVQYLERFSARVLSPLDFRRYPFDSQTLHIYLIVRSVDTRNIVLAVDLEKVGKNDDVFLTGW
                                                                                
--------------------------------------------------------------------------------


DIESFTAVVKPANFALEDRLESKLDYQLRISRQYFSYIPNIILPMLFILFISWTAFWSTSYEANVTLVVSTLIAHIAFNI
                                                                                
--------------------------------------------------------------------------------


LVETNLPKTPYMTYTGAIIFMIYLFYFVAVIEVTVQHYLKVESQPARAASITRASRIAFPVVFLLANIILAFLFFGF
                       ******************************************************
-----------------------LFYFVAVIEVTVQHYLKVESQPARAASITRASRIAFPVVFLLANIILAFLFFGF


Identity seq1: 17.03%
Identit

In [103]:
seq_1 = 'AQDMVSP'
seq_2 = 'AQDMVSPPPPIA'

from pdb_numpy.data.blosum import BLOSUM62

def align_seq_DTW(seq_1, seq_2):
    """ Align two amino acid sequences using the Dynamic Time Warping Algorithm.
    """

    seq_1 = seq_1.replace("-", "")
    seq_2 = seq_2.replace("-", "")

    len_1 = len(seq_1)
    len_2 = len(seq_2)

    # Initialize the matrix
    matrix = np.zeros((len_1 + 1, len_2 + 1))
    matrix[1:, 0] = -np.inf
    matrix[0, 1:] = -np.inf

    direction = np.zeros((len_1 + 1, len_2 + 1))

    choices = np.zeros(3)

    # Fill the matrix
    for i in range(1, len_1 + 1):
        # print(i)
        prev = False  # insertion matrix[i, j - 1]
        for j in range(1, len_2 + 1):
            # Identify the BLOSUM62 score
            # Match
            choices = np.array([matrix[i - 1, j - 1], matrix[i - 1, j],  matrix[i, j - 1]])

            max_index = np.argmax(choices)
            #print(max_index)
            matrix[i, j] = choices[max_index] + BLOSUM62[(seq_2[j - 1], seq_1[i - 1])]
            direction[i, j] = max_index
    
    print(matrix)
    print(direction)

    i = len_1
    j = len_2

    align_1 = ""
    align_2 = ""

    while i != 0 and j != 0:
        if (
            direction[i, j] == 0
        ):
            align_1 = seq_1[i - 1] + align_1
            align_2 = seq_2[j - 1] + align_2
            i -= 1
            j -= 1
        elif (
            direction[i, j] == 1
        ):
            align_1 = seq_1[i - 1] + align_1
            align_2 = "-" + align_2
            i -= 1
        elif (
            direction[i, j] == 2
        ):
            align_1 = "-" + align_1
            align_2 = seq_2[j - 1] + align_2
            j -= 1

    i = len(align_1) - 1
    while i > 0:
        if (align_1[i] == "-" and align_2[i -1] == "-"):
            align_1 = align_1[:i] + align_1[i + 1:]
            align_2 = align_2[:i-1] + align_2[i:]
        elif (align_2[i] == "-" and align_1[i -1] == "-"):
            align_2 = align_2[:i] + align_2[i + 1:]
            align_1 = align_1[:i-1] + align_1[i:]
        i -= 1

    assert len(align_1) == len(align_2)

    print(align_1)
    print(align_2)

    return align_1, align_2

    

In [126]:
seq_1 = "AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV"
seq_2 = "AQDMVSPPPPIADEPLTVN"
align_seq_DTW(seq_1, seq_2)

[[  0. -inf -inf ... -inf -inf -inf]
 [-inf   4.   3. ...  -5.  -5.  -7.]
 [-inf   3.   9. ...  -2.  -4.  -4.]
 ...
 [-inf -29. -25. ...  95.  96. 100.]
 [-inf -30. -26. ...  94.  94.  98.]
 [-inf -30. -28. ...  94.  98.  95.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 2. ... 2. 2. 2.]
 [0. 1. 0. ... 2. 2. 2.]
 ...
 [0. 1. 1. ... 1. 1. 0.]
 [0. 1. 1. ... 1. 1. 1.]
 [0. 1. 1. ... 1. 0. 1.]]
AQDMVSP--PPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV
AQDMVSPPPP--IADEPLT----------------------------------V--N--


('AQDMVSP--PPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV',
 'AQDMVSPPPP--IADEPLT----------------------------------V--N--')

In [127]:
import numpy as np
from tmtools import tm_align

In [128]:
coords1 = pdb_numpy.Coor("src/pdb_numpy/tests/input/1u85.pdb")
coords2 = pdb_numpy.Coor("src/pdb_numpy/tests/input/1ubd.pdb")

coords1_ca = coords1.select_atoms("name CA")
coords2_ca = coords2.select_atoms("name CA")

seq1 = coords1.get_aa_seq()['A']
seq2 = coords2.get_aa_seq()['C']
res = tm_align(coords1_ca.xyz, coords2_ca.xyz, seq1, seq2)

In [131]:
%timeit tm_align(coords1_ca.xyz, coords2_ca.xyz, seq1, seq2)

4.22 ms ± 22.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [129]:
coords2.get_aa_seq()

{'C': 'TIACPHKGCTKMFRDNSAMRKHLHTHGPRVHVCAECGKAFVESSKLKRHQLVHTGEKPFQCTFEGCGKRFSLDFNLRTHVRIHTGDRPYVCPFDGCNKKFAQSTNLKSHILTHA'}

In [130]:
res.u

array([[-0.75530653, -0.5759304 ,  0.31275584],
       [ 0.17315727,  0.28488949,  0.94279082],
       [-0.63208275,  0.76625202, -0.11545235]])